# Changes from v1

* Explore the data more to see how many of each CPT code there are
* Run tf-idf with max_df = .7
* Crop any CPT codes with less than 1000 occurrences in the dataset

# Import the MIMIC data

In [1]:
import gzip
import pandas as pd
import glob

dataset_dictionary = {}

for file_path in glob.glob('.\\Data\\MIMIC Files\*'):
    file_name = file_path.split('\\')[3].split('.')[0]
    with gzip.open(file_path, mode='r') as file:
        dataset_dictionary[file_name] = pd.read_csv(file)

c:\users\amartins\onedrive - intermountain healthcare\python_pycharm_virt_env\.venv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\users\amartins\onedrive - intermountain healthcare\python_pycharm_virt_env\.venv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Join the tables

In [2]:
# Create dataset to join together -----

# Create note_events table -----

# Combine text for each subject and encounter
note_events_base = dataset_dictionary['NOTEEVENTS'][dataset_dictionary['NOTEEVENTS'].loc[:,'CATEGORY'] == 'Discharge summary']
note_events = note_events_base.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False)['TEXT'].agg(sum)

# Create CPT table -----

cpt_events_base = dataset_dictionary['CPTEVENTS'].loc[:, ['SUBJECT_ID','HADM_ID', 'CPT_CD']]
cpt_events = cpt_events_base.drop_duplicates()

# Join the datasets -----

note_cpt = note_events.merge(cpt_events, on = ['SUBJECT_ID','HADM_ID'])
# print(note_cpt.shape, note_events.shape, cpt_events.shape) # (223,150, 4) (52,726, 3) (227,510, 3)

# Filter the data to CPT with over 1000 instances

In [63]:
# Find CPT codes occurring over 1000 times and put into a list
cpt_1000 = note_cpt['CPT_CD'].astype(str).value_counts(ascending=False) >= 1000
cpt_1000_ls = list(cpt_1000[cpt_1000].index)

note_cpt_1000 = note_cpt[note_cpt['CPT_CD'].astype(str).isin(cpt_1000_ls)]

# Split the Data

In [67]:
# Import Packages -----

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Split the data -----

X_train, X_test, y_train, y_test = train_test_split(note_cpt_1000['TEXT'], note_cpt_1000['CPT_CD'].astype(str), test_size = .33, random_state = 42)


# Tokenize the data

In [68]:
# Tokenize the data -----

# Import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df = .7)

# Transform the training data
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data
tfidf_test = tfidf_vectorizer.transform(X_test)

# print(tfidf_vectorizer.get_feature_names()[:10])

# Run Naive Bayes

In [69]:
# Use Naive Bayes model -----

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nb_classifier = MultinomialNB()

# Fit and check accuracy
nb_classifier.fit(tfidf_train, y_train)
pred = nb_classifier.predict(tfidf_test)

In [73]:
metrics.accuracy_score(y_test, pred)

# """
# V1 NLP Model Accuracy: 0.117
# Wow, I've got a long way to go to improve accuracy
# V2 NLP Model Accuracy: 0.14
# """

# Confusion matrix 
# confusion_mtrx = metrics.confusion_matrix(y_test.astype(str), pred) # 1380, 1380
# confusion_mtrx

0.14178087812884693